# Create and monitor copy collection jobs in Azure Cosmos DB SQL API account

## Login to the Subscription.

In [ ]:
az login

## Set a subscription to be the current active subscription

(needed only if you multiple subscriptions associated with your account)

In [ ]:
az account set --subscription ""

## Install the Cosmos DB preview CLI  extension that contains the container copy commands.

In [ ]:
az extension add --name cosmosdb-preview

## Specify parameters necessary for the copy

In [ ]:
$cosmosdbaccountname = "cosmosDBAccountName"
$rg = "resourceGroupName"
$copyJobName = ""
$sourceDb = ""
$sourceContainer = ""
$destinationDb = ""
$destinationContainer = ""

## Create container copy job

In [ ]:
az cosmosdb dts copy -g $rg --job-name $copyJobName --account-name $cosmosdbaccountname \
    --source-sql-container database=$sourceDb container=$sourceContainer \
    --dest-sql-container database=$destinationDb container=$destinationContainer

## Monitor the progress of the container copy job

In [ ]:
az cosmosdb dts show --account-name  $cosmosdbaccountname -g $rg --job-name $copyJobname